In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.neighbors import NearestNeighbors
import seaborn as sns

# Finding counterfactuals

In [3]:
num = 100000
Z = np.random.normal(0, 1, num)
Z_prob = 1 / (1 + np.exp(Z))
X = np.random.binomial(1, Z_prob)
Y = 2 * X + 3 * X * Z + Z + np.random.normal(0, 1, num)
df = pd.DataFrame({"X": X, "Z": Z, "Y": Y, "X_Z": X * Z})

In [39]:
num = 100000
Z = np.random.normal(0, 1, num)
Z_prob = 1 / (1 + np.exp(Z))
X = np.random.binomial(1, Z_prob)
Y = 2 * X + 3 * X * Z + Z + np.random.normal(0, 1, num)
df = pd.DataFrame({"X": X, "Z": Z, "Y": Y, "X_Z": X * Z})

In [40]:
results = sm.OLS(Y, sm.add_constant(df.drop(columns=["Y"]))).fit()
results.params

const   -0.001657
X        2.009576
Z        0.997907
X_Z      3.006594
dtype: float64

In [41]:
treated = df[df.X == 1]
untreated = df[df.X == 0]
print(untreated.Y.mean())
print(treated.Y.mean())

0.40660999546062243
0.3173804251417051


In [42]:
# ATT
treated_cf_Y = results.predict(pd.DataFrame({"const": 1, "X": 0, "Z": treated.Z, "X_Z": 0}))
treated.Y.mean() - treated_cf_Y.mean()

0.7403136204672914

In [43]:
2 + 3 * treated.Z.mean() # Estimate of ATT, Y at X = 1 minus Y at X = 0 (the Z is present in both, so it cancels)

0.7335211037521083

In [44]:
#ATUT
untreated_cf_Y = results.predict(pd.DataFrame({"const": 1, "X": 1, "Z": untreated.Z, "X_Z": untreated.Z}))
untreated_cf_Y.mean() - untreated.Y.mean()

3.2396446741237326

In [45]:
2 + 3 * untreated.Z.mean() # Estimate of ATUT, Y at X = 1 minus Y at X = 0

3.227370992049526

In [46]:
#ATE
((treated.Y.mean() - treated_cf_Y.mean()) * treated.shape[0] + (untreated_cf_Y.mean() - untreated.Y.mean()) * untreated.shape[0]) \
/ df.shape[0]

1.9947528696079957

In [47]:
2 + 3 * df.Z.mean() # Estimate of ATE

1.9852093011874652

In [53]:
num = 100000
Z_mean = 0.5
Z = np.random.normal(Z_mean, 1, num)
Z_prob = 1 / (1 + np.exp(Z))
X = np.random.binomial(1, Z_prob)
Y = 2 * X + Z + np.random.normal(0, 1, num)
df = pd.DataFrame({"X": X, "Y": Y, "Z": Z, "X_Z": X * Z})

### Linear regression for prediction: without X * Z

In [55]:
model = sm.OLS(Y, sm.add_constant(df[["X", "Z"]]))
results = model.fit()
results.params

const    0.004785
X        1.992043
Z        1.000669
dtype: float64

In [56]:
treated = df[df.X == 1]
untreated = df[df.X == 0]
treated_cf = sm.add_constant(pd.DataFrame({"X": 0, "Z": treated["Z"]}), has_constant = "add")
treated_cf_prediction = results.predict(treated_cf)

In [59]:
np.vstack((treated.Y, treated_cf_prediction)).T

array([[ 1.58633329,  0.30814614],
       [ 1.36327739, -0.32031375],
       [ 0.19571861, -2.13835599],
       ...,
       [ 4.69515385,  0.54638205],
       [ 3.83086203,  0.70402237],
       [ 3.80925195,  0.74907433]])

In [60]:
treated.Y.mean() - treated_cf_prediction.mean()

1.9920430185896687

In [59]:
treated = df[df.X == 1]
untreated = df[df.X == 0]

treated_cf = sm.add_constant(pd.DataFrame({"X": 0, "Z": treated["Z"]}), has_constant = "add")
treated_cf_prediction = results.predict(treated_cf)
untreated_cf = sm.add_constant(pd.DataFrame({"X": 1, "Z": untreated["Z"]}), has_constant = "add")
untreated_cf_prediction = results.predict(untreated_cf)

print("ATT effect:", treated.Y.mean() - treated_cf_prediction.mean())
print("ATUT effect:", untreated_cf_prediction.mean() - untreated.Y.mean())
print("ATE effect:", ((treated.Y.mean() - treated_cf_prediction.mean()) * treated.shape[0] + (untreated_cf_prediction.mean() - untreated.Y.mean()) * untreated.shape[0]) / (untreated.shape[0] + treated.shape[0]))

ATT effect: 2.00431624396385
ATUT effect: 2.004316243963347
ATE effect: 2.004316243963548


In [61]:
num = 100000
Z_mean = 0.5
Z = np.random.normal(Z_mean, 1, num)
Z_prob = 1 / (1 + np.exp(Z))
X = np.random.binomial(1, Z_prob)
Y = 2 * X + 3 * X * Z + Z + np.random.normal(0, 1, num)
df = pd.DataFrame({"X": X, "Y": Y, "Z": Z, "X_Z": X * Z})

### Linear regression for prediction: with X * Z

In [62]:
model = sm.OLS(Y, sm.add_constant(df[["X", "Z", "X_Z"]]))
results = model.fit()
results.params

const    0.003806
X        1.998134
Z        0.999543
X_Z      2.994023
dtype: float64

In [23]:
treated = df[df.X == 1]
untreated = df[df.X == 0]

treated_cf = sm.add_constant(pd.DataFrame({"X": 0, "Z": treated["Z"], "X_Z": 0}), has_constant = "add")
treated_cf_prediction = results.predict(treated_cf)
untreated_cf = sm.add_constant(pd.DataFrame({"X": 1, "Z": untreated["Z"], "X_Z": untreated["Z"]}), has_constant = "add")
untreated_cf_prediction = results.predict(untreated_cf)

print("treated Z mean:", treated.Z.mean(), "2 + 3 * Z = ", 2 + 3 * treated.Z.mean())
print("ATT effect:", treated.Y.mean() - treated_cf_prediction.mean())
print("untreated Z mean:", untreated.Z.mean(), "2 + 3 * Z = ", 2 + 3 * untreated.Z.mean())
print("ATUT effect:", untreated_cf_prediction.mean() - untreated.Y.mean())
print("overall Z mean:", df.Z.mean(), "2 + 3 * Z = ", 2 + 3 * df.Z.mean())
print("ATE effect:", ((treated.Y.mean() - treated_cf_prediction.mean()) * treated.shape[0] + (untreated_cf_prediction.mean() - untreated.Y.mean()) * untreated.shape[0]) / (untreated.shape[0] + treated.shape[0]))

treated Z mean: -0.00601439129669435 2 + 3 * Z =  1.981956826109917
ATT effect: 1.992310724689152
untreated Z mean: 0.8238457102294724 2 + 3 * Z =  4.471537130688417
ATUT effect: 4.4678040706944655
overall Z mean: 0.4919182668210362 2 + 3 * Z =  3.4757548004631085
ATE effect: 3.4776562421592603


### An unweighted ATE effect differs from the correct ATE effect

In [25]:
# Note that if you don't weight by the number of items in treated and untreated, you get the wrong result
print("ATE effect (wrong):", ((treated.Y.mean() - treated_cf_prediction.mean()) + (untreated_cf_prediction.mean() - untreated.Y.mean())) / 2)

ATE effect (wrong): 3.2300573976918088


### Optimal Treatment Effect (In the homework I called this Marginal, but it's not quite the right term)

In [26]:
arg = (untreated_cf_prediction.values - untreated.Y.values).argmax()
print(untreated_cf_prediction.iloc[arg])
print(untreated.iloc[arg])

22.71605166781269
X      0.000000
Y      3.756040
Z      5.186843
X_Z    0.000000
Name: 12250, dtype: float64


In [27]:
results.params

const   -0.011338
X        2.010252
Z        1.011146
X_Z      2.983025
dtype: float64

### Matching for prediction

In [43]:
neighbors = NearestNeighbors(n_neighbors = 1).fit(untreated[["Z"]])
distances, indices = neighbors.kneighbors(treated[["Z"]])
treated.Y.mean() - untreated.iloc[indices.ravel()].Y.mean()

1.9849456525319045

In [44]:
neighbors = NearestNeighbors(n_neighbors = 1).fit(treated[["Z"]])
distances, indices = neighbors.kneighbors(untreated[["Z"]])
treated.iloc[indices.ravel()].Y.mean() - untreated.Y.mean()

4.475828329322422

In [45]:
neighbors = NearestNeighbors(n_neighbors = 1).fit(untreated[["Z"]])
distances, indices = neighbors.kneighbors(treated[["Z"]])
treated.Y.mean() - untreated.iloc[indices.ravel()].Y.mean()

1.9849456525319045

In [46]:
neighbors = NearestNeighbors(n_neighbors = 1).fit(untreated[["Z"]])
distances, indices = neighbors.kneighbors(treated[["Z"]])
treated_cf = untreated.iloc[indices.ravel()]

In [47]:
neighbors = NearestNeighbors(n_neighbors = 1).fit(treated[["Z"]])
distances, indices = neighbors.kneighbors(untreated[["Z"]])
untreated_cf = treated.iloc[indices.ravel()]

In [48]:
print("ATT effect:", treated.Y.mean() - treated_cf.Y.mean())
print("ATUT effect:", untreated_cf.Y.mean() - untreated.Y.mean())
print("ATE effect:", ((treated.Y.mean() - treated_cf.Y.mean()) * treated.shape[0] + (untreated_cf.Y.mean() - untreated.Y.mean()) * untreated.shape[0]) / (untreated.shape[0] + treated.shape[0]))

ATT effect: 1.9849456525319045
ATUT effect: 4.475828329322422
ATE effect: 3.4795250762597507


### Optimal Treatment Effect

In [30]:
arg = (untreated_cf.reset_index().Y - untreated.reset_index().Y).argmax()
print(untreated_cf.iloc[arg])
print(untreated.iloc[arg])

X       1.000000
Y      15.222356
Z       3.940958
X_Z     3.940958
Name: 18587, dtype: float64
X      0.000000
Y     -2.319297
Z      4.315572
X_Z    0.000000
Name: 4544, dtype: float64
